## Tanzania water well challenge
Please fill out:

Student name: Austin Murray
Student pace: full time
Scheduled project review date/time:
Instructor name: Yish
Blog post URL:

## Overview
Drivendata.org is having a competition with data about the waterwells in Tazania to see how accurately one can predict if a well is functioning, not functioning or functioning but needs repaired.

## The Data
The data is from the Pump it Up challenge on drivendata.org

This is geographical, governmental, and functional data about over 59000 water wells in Tanzania

## Importing all the tools

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium
import requests
from bs4 import BeautifulSoup


In [12]:
pip install cfbd

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [13]:
import cfbd 

In [14]:
from __future__ import print_function
import time
import cfbd
from cfbd.rest import ApiException
from pprint import pprint

In [15]:
# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Bearer'] = 'XqWTKOTB82w8/rJv0DKldJyj7kwU7HNK8ddvoVZCGnxiuxqVnHswS9m5GKqouNN1'

In [29]:
players_api = cfbd.PlayersApi()
production = players_api.get_returning_production(year=2019)
production_df = pd.DataFrame.from_records([p.to_dict() for p in production])
production_df.head()

,season,team,conference,total_ppa,total_passing_ppa,total_receiving_ppa,total_rushing_ppa,percent_ppa,percent_passing_ppa,percent_receiving_ppa,percent_rushing_ppa,usage,passing_usage,receiving_usage,rushing_usage
0,2019,Air Force,Mountain West,206.4,76.3,33.4,96.7,0.653,0.959,0.322,0.729,0.624,0.885,0.393,0.615
1,2019,Akron,Mid-American,144.5,33.6,111.0,-0.1,0.855,1.000,0.737,0.006,0.704,1.000,0.639,0.442
2,2019,Alabama,SEC,467.2,180.3,247.6,39.3,0.584,0.608,0.708,0.256,0.434,0.455,0.593,0.303
3,2019,Appalachian State,Sun Belt,340.2,92.4,167.0,80.8,0.840,0.964,0.867,0.692,0.844,0.923,0.837,0.793
4,2019,Arizona,Pac-12,218.9,100.2,62.0,56.7,0.587,0.947,0.307,0.871,0.793,0.994,0.356,0.944


In [123]:
production

[{'conference': 'Mountain West',
  'passing_usage': 0.885,
  'percent_passing_ppa': 0.959,
  'percent_ppa': 0.653,
  'percent_receiving_ppa': 0.322,
  'percent_rushing_ppa': 0.729,
  'receiving_usage': 0.393,
  'rushing_usage': 0.615,
  'season': 2019,
  'team': 'Air Force',
  'total_passing_ppa': 76.3,
  'total_ppa': 206.4,
  'total_receiving_ppa': 33.4,
  'total_rushing_ppa': 96.7,
  'usage': 0.624},
 {'conference': 'Mid-American',
  'passing_usage': 1.0,
  'percent_passing_ppa': 1.0,
  'percent_ppa': 0.855,
  'percent_receiving_ppa': 0.737,
  'percent_rushing_ppa': 0.006,
  'receiving_usage': 0.639,
  'rushing_usage': 0.442,
  'season': 2019,
  'team': 'Akron',
  'total_passing_ppa': 33.6,
  'total_ppa': 144.5,
  'total_receiving_ppa': 111.0,
  'total_rushing_ppa': -0.1,
  'usage': 0.704},
 {'conference': 'SEC',
  'passing_usage': 0.455,
  'percent_passing_ppa': 0.608,
  'percent_ppa': 0.584,
  'percent_receiving_ppa': 0.708,
  'percent_rushing_ppa': 0.256,
  'receiving_usage': 0.59

In [52]:
recruiting_api = cfbd.RecruitingApi()

player_recruit_stats = []

recruit_years = range(2000, 2015)
for i in recruit_years:
    
    player_recruit_stats = player_recruit_stats + recruiting_api.get_recruiting_players(year=i)
    
player_recruit_stats_df = pd.DataFrame.from_records([p.to_dict() for p in player_recruit_stats])
player_recruit_stats_df.head()

,recruit_type,year,ranking,name,school,committed_to,position,height,weight,stars,rating,city,state_province,country
0,HighSchool,2000,1,D.J. Williams,De La Salle,Miami,ILB,74.0,235.0,5,0.9998,Concord,CA,USA
1,HighSchool,2000,2,Brock Berlin,Evangel Christian Academy,Florida,PRO,74.0,190.0,5,0.9998,Shreveport,LA,USA
2,HighSchool,2000,3,Charles Rogers,Saginaw,Michigan State,WR,76.0,195.0,5,0.9988,Saginaw,MI,USA
3,HighSchool,2000,4,Travis Johnson,Notre Dame,Florida State,SDE,76.0,265.0,5,0.9982,Sherman Oaks,CA,USA
4,HighSchool,2000,5,Marcus Houston,Thomas Jefferson,Colorado,RB,72.0,208.0,5,0.9980,Denver,CO,USA


In [206]:
duplicatedrecruitDF = player_recruit_stats_df[player_recruit_stats_df.duplicated(['name', 'committed_to'],keep=False)]

In [207]:
duplicatedrecruitDF.head(50)

,recruit_type,year,ranking,name,school,committed_to,position,height,weight,stars,rating,city,state_province,country


Since the only difference is the rating we will go wtih their higher rating.

In [200]:
player_recruit_stats_df.drop_duplicates(subset=['name', 'committed_to','year'],keep='first', inplace=True) #to keep their higher rating

In [205]:
player_recruit_stats_df.drop_duplicates(subset=['name', 'committed_to'],keep='last', inplace=True) #some players were rated twice in two different years so we kept the most recent

In [208]:
player_recruit_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32783 entries, 0 to 32893
Data columns (total 14 columns):
recruit_type      32783 non-null object
year              32783 non-null int64
ranking           32783 non-null int64
name              32783 non-null object
school            32783 non-null object
committed_to      30421 non-null object
position          32783 non-null object
height            32669 non-null float64
weight            32666 non-null float64
stars             32783 non-null int64
rating            32783 non-null float64
city              32779 non-null object
state_province    32779 non-null object
country           32779 non-null object
dtypes: float64(3), int64(3), object(8)
memory usage: 3.8+ MB


In [209]:
player_recruit_stats_df.year.value_counts()

2014    3842
2013    3469
2012    3099
2011    2678
2010    2467
2009    2288
2007    2218
2008    2169
2005    2165
2006    2152
2004    2127
2003    2104
2002    1757
2001     128
2000     120
Name: year, dtype: int64

In [211]:
player_recruit_stats_df = player_recruit_stats_df.drop(['state_province','country','recruit_type'], axis = 1)

In [214]:
player_recruit_stats_df = player_recruit_stats_df.drop(['city'], axis = 1)

In [215]:
player_recruit_stats_df

,year,ranking,name,school,committed_to,position,height,weight,stars,rating
0,2000,1,D.J. Williams,De La Salle,Miami,ILB,74.0,235.0,5,0.9998
1,2000,2,Brock Berlin,Evangel Christian Academy,Florida,PRO,74.0,190.0,5,0.9998
2,2000,3,Charles Rogers,Saginaw,Michigan State,WR,76.0,195.0,5,0.9988
3,2000,4,Travis Johnson,Notre Dame,Florida State,SDE,76.0,265.0,5,0.9982
4,2000,5,Marcus Houston,Thomas Jefferson,Colorado,RB,72.0,208.0,5,0.9980
...,...,...,...,...,...,...,...,...,...,...
32889,2014,3808,Christian Triplett,Delano Senior,North Dakota State,DT,77.0,270.0,1,0.6997
32890,2014,3809,Austin Stapleton,St Joseph Regional,Pennsylvania,OT,77.0,290.0,1,0.6997
32891,2014,3810,Brandon Taylor,Brentwood Academy,Houston Baptist,APB,69.0,155.0,1,0.6997
32892,2014,3811,Jorge Powell,Belen Jesuit Prep,Florida,K,70.0,160.0,1,0.6997


In [216]:
player_recruit_stats_df.rename(columns = {'committed_to':'team'}, inplace = True)

In [223]:
player_recruit_stats_df.rename(columns = {'name':'player'}, inplace = True)

In [59]:

player_season_stats = []

season_years = range(1997, 2015)
for i in season_years:
    
    player_season_stats = player_season_stats + players_api.get_player_season_stats(year=i)
    
player_season_stats_df = pd.DataFrame.from_records([p.to_dict() for p in player_season_stats])
player_season_stats_df.head()


,season,player_id,player,team,conference,category,stat_type,stat
0,None,147939,Michael LeDet,Tulsa,Western Athletic,interceptions,TD,0.0
1,None,146357,Donnie Johnson,Penn State,Big Ten,puntReturns,LONG,6.0
2,None,134775,Trent Shelton,Baylor,Big 12,receiving,YDS,426.0
3,None,166666,Jae Thaxton,Florida State,ACC,puntReturns,AVG,15.0
4,None,136528,Eddie Jackson,West Virginia,Big East,receiving,YDS,246.0


In [60]:
player_season_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325998 entries, 0 to 325997
Data columns (total 8 columns):
season        0 non-null object
player_id     325998 non-null int64
player        325998 non-null object
team          325998 non-null object
conference    325998 non-null object
category      325998 non-null object
stat_type     325998 non-null object
stat          325998 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 19.9+ MB


In [116]:
player_season_stats_df = player_season_stats_df.drop(['season'], axis=1)

In [117]:
player_season_stats_df['stat_category'] = player_season_stats_df['category'] + player_season_stats_df['stat_type']

In [118]:
player_season_stats_df.head()

,player_id,player,team,conference,category,stat_type,stat,stat_category
0,147939,Michael LeDet,Tulsa,Western Athletic,interceptions,TD,0.0,interceptionsTD
1,146357,Donnie Johnson,Penn State,Big Ten,puntReturns,LONG,6.0,puntReturnsLONG
2,134775,Trent Shelton,Baylor,Big 12,receiving,YDS,426.0,receivingYDS
3,166666,Jae Thaxton,Florida State,ACC,puntReturns,AVG,15.0,puntReturnsAVG
4,136528,Eddie Jackson,West Virginia,Big East,receiving,YDS,246.0,receivingYDS


In [121]:
player_season_stats_df = player_season_stats_df[["player_id", "player", "team", "conference", "category", "stat_type", "stat_category", "stat"]]

In [122]:
player_season_stats_df.head()

,player_id,player,team,conference,category,stat_type,stat_category,stat
0,147939,Michael LeDet,Tulsa,Western Athletic,interceptions,TD,interceptionsTD,0.0
1,146357,Donnie Johnson,Penn State,Big Ten,puntReturns,LONG,puntReturnsLONG,6.0
2,134775,Trent Shelton,Baylor,Big 12,receiving,YDS,receivingYDS,426.0
3,166666,Jae Thaxton,Florida State,ACC,puntReturns,AVG,puntReturnsAVG,15.0
4,136528,Eddie Jackson,West Virginia,Big East,receiving,YDS,receivingYDS,246.0


In [152]:
bcc = player_season_stats_df.loc[player_season_stats_df['player'] == 'Dion Lewis']

In [153]:
bcc

,player_id,player,team,conference,category,stat_type,stat_category,stat
141672,482840,Dion Lewis,Pittsburgh,Big East,rushing,LONG,rushingLONG,85.0
144138,482840,Dion Lewis,Pittsburgh,Big East,receiving,YPR,receivingYPR,7.6
152005,482840,Dion Lewis,Pittsburgh,Big East,receiving,TD,receivingTD,1.0
152276,482840,Dion Lewis,Pittsburgh,Big East,rushing,TD,rushingTD,17.0
152927,482840,Dion Lewis,Pittsburgh,Big East,receiving,YDS,receivingYDS,189.0
154696,482840,Dion Lewis,Pittsburgh,Big East,rushing,CAR,rushingCAR,325.0
155124,482840,Dion Lewis,Pittsburgh,Big East,receiving,REC,receivingREC,25.0
156497,482840,Dion Lewis,Pittsburgh,Big East,rushing,YDS,rushingYDS,1799.0
158099,482840,Dion Lewis,Pittsburgh,Big East,receiving,LONG,receivingLONG,31.0
163104,482840,Dion Lewis,Pittsburgh,Big East,rushing,YPC,rushingYPC,5.5


In [155]:
bdd = bcc.groupby(['player_id', 'player', 'team', 'stat_category'])['stat'].mean()

In [156]:
bdd

player_id  player      team        stat_category
482840     Dion Lewis  Pittsburgh  receivingLONG      31.00
                                   receivingREC       26.00
                                   receivingTD         0.50
                                   receivingYDS      202.50
                                   receivingYPR        7.80
                                   rushingCAR        272.00
                                   rushingLONG        80.50
                                   rushingTD          15.00
                                   rushingYDS       1430.00
                                   rushingYPC          5.15
Name: stat, dtype: float64

In [158]:
gkk = player_season_stats_df.groupby(['player_id', 'team', 'player','stat_category'])['stat'].mean()

In [159]:
gkk.head(50)

player_id  team           player  stat_category     
-7111      Pittsburgh      Team   puntReturnsAVG       -3.0
                                  puntReturnsLONG       0.0
                                  puntReturnsNO         1.0
                                  puntReturnsTD         0.0
                                  puntReturnsYDS       -3.0
                                  rushingCAR            1.0
                                  rushingLONG           0.0
                                  rushingTD             0.0
                                  rushingYDS           -4.0
                                  rushingYPC           -4.0
-7088      Penn State      Team   rushingCAR            2.0
                                  rushingLONG           0.0
                                  rushingTD             0.0
                                  rushingYDS           -7.0
                                  rushingYPC           -3.5
-7014      Ohio State      Team   rushingCAR   

In [160]:
gkk = gkk.unstack(level=-1)

In [161]:
gkk

,,stat_category,interceptionsAVG,interceptionsINT,interceptionsTD,interceptionsYDS,kickReturnsAVG,kickReturnsLONG,kickReturnsNO,kickReturnsTD,kickReturnsYDS,kickingFGA,...,receivingLONG,receivingREC,receivingTD,receivingYDS,receivingYPR,rushingCAR,rushingLONG,rushingTD,rushingYDS,rushingYPC
player_id,team,player,,,,,,,,,,,,,,,,,,,,,
-7111,Pittsburgh,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-4.0,-4.0
-7088,Penn State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-7.0,-3.5
-7014,Ohio State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-1.0,-1.0
-6965,Notre Dame,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-4.0,-2.0
-6810,Navy,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3159129,Middle Tennessee,Sean O'kane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3160510,Arkansas State,Logan Spry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3161453,Arizona,- Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,-6.0,-1.5


In [162]:
gkk.reset_index()

stat_category,player_id,team,player,interceptionsAVG,interceptionsINT,interceptionsTD,interceptionsYDS,kickReturnsAVG,kickReturnsLONG,kickReturnsNO,...,receivingLONG,receivingREC,receivingTD,receivingYDS,receivingYPR,rushingCAR,rushingLONG,rushingTD,rushingYDS,rushingYPC
0,-7111,Pittsburgh,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-4.0,-4.0
1,-7088,Penn State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-7.0,-3.5
2,-7014,Ohio State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-1.0,-1.0
3,-6965,Notre Dame,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-4.0,-2.0
4,-6810,Navy,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20027,3159129,Middle Tennessee,Sean O'kane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20028,3160510,Arkansas State,Logan Spry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20029,3161453,Arizona,- Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,-6.0,-1.5
20030,3164432,Maryland,- Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
gkk.to_csv('season_avg_stats.csv')

In [171]:
season_avg_stats_df = pd.read_csv('season_avg_stats.csv')

In [173]:
season_avg_stats_df

,player_id,team,player,interceptionsAVG,interceptionsINT,interceptionsTD,interceptionsYDS,kickReturnsAVG,kickReturnsLONG,kickReturnsNO,...,receivingLONG,receivingREC,receivingTD,receivingYDS,receivingYPR,rushingCAR,rushingLONG,rushingTD,rushingYDS,rushingYPC
0,-7111,Pittsburgh,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-4.0,-4.0
1,-7088,Penn State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-7.0,-3.5
2,-7014,Ohio State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-1.0,-1.0
3,-6965,Notre Dame,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-4.0,-2.0
4,-6810,Navy,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20027,3159129,Middle Tennessee,Sean O'kane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20028,3160510,Arkansas State,Logan Spry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20029,3161453,Arizona,- Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,-6.0,-1.5
20030,3164432,Maryland,- Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
season_avg_stats_df.player.value_counts()

- Team              17
Brandon Williams     9
Michael Johnson      9
 Team                9
Chris Johnson        8
                    ..
Chaz Walker          1
Darius Barnes        1
Neal Howey           1
Tyler Boyd           1
Jared Fagan          1
Name: player, Length: 18888, dtype: int64

In [180]:
season_avg_stats_df.loc[season_avg_stats_df['player'] == ' Team']

,player_id,team,player,interceptionsAVG,interceptionsINT,interceptionsTD,interceptionsYDS,kickReturnsAVG,kickReturnsLONG,kickReturnsNO,...,receivingLONG,receivingREC,receivingTD,receivingYDS,receivingYPR,rushingCAR,rushingLONG,rushingTD,rushingYDS,rushingYPC
0,-7111,Pittsburgh,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-4.0,-4.0
1,-7088,Penn State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-7.0,-3.5
2,-7014,Ohio State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-1.0,-1.0
3,-6965,Notre Dame,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-4.0,-2.0
4,-6810,Navy,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-6530,Marshall,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,-9.0,-3.0
6,-5995,Florida State,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,-6.0,-3.0
7,-5151,Akron,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,-4.0,-1.3
8,-5,UAB,Team,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.0,0.0,0.0,-9.0,-1.5


no reason to delete the team rows as they will be taken out when we merge and there is no player named team.

In [222]:
duplicatedrecruitDF = season_avg_stats_df[season_avg_stats_df.duplicated(['player', 'committed_to'],keep=False)]

AttributeError: 'str' object has no attribute 'type'

In [224]:
recruit_and_season_avg_df = pd.merge(player_recruit_stats_df, season_avg_stats_df, on=["player", "team"])

In [225]:
recruit_and_season_avg_df

,year,ranking,player,school,team,position,height,weight,stars,rating,...,receivingLONG,receivingREC,receivingTD,receivingYDS,receivingYPR,rushingCAR,rushingLONG,rushingTD,rushingYDS,rushingYPC
0,2000,8,Bryant McFadden,McArthur,Florida State,CB,73.0,182.0,5,0.9968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,17,Chance Mock,The Woodlands,Texas,PRO,74.0,190.0,5,0.9918,...,NaN,NaN,NaN,NaN,NaN,7.0,3.0,0.0,-14.0,-2.0
2,2000,19,Matt Grootegoed,Mater Dei,USC,OLB,71.0,205.0,5,0.9914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,28,Cory Bailey,American,Florida,S,70.0,175.0,5,0.9880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,32,Jason Campbell,Taylorsville,Auburn,PRO,77.0,190.0,5,0.9853,...,NaN,NaN,NaN,NaN,NaN,58.0,28.0,3.0,30.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,2014,3466,Papi White,Seminole,Ohio,RB,69.0,168.0,2,0.7330,...,7.0,1.0,0.0,7.0,7.0,6.0,7.0,0.0,23.0,3.8
10458,2014,3732,Christian Hagan,Westside,Northern Illinois,K,72.0,165.0,1,0.6997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10459,2014,3736,Brett Layton,Redlands,Ohio,CB,71.0,190.0,1,0.6997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10460,2014,3781,Hunter Sharp,Antelope Valley,Utah State,WR,73.0,190.0,1,0.6997,...,81.0,66.0,7.0,939.0,14.2,6.0,12.0,0.0,28.0,4.7


In [250]:
recruit_and_season_avg_df.loc[recruit_and_season_avg_df['player'] == 'Derrick Johnson']

,year,ranking,player,school,team,position,height,weight,stars,rating,...,receivingLONG,receivingREC,receivingTD,receivingYDS,receivingYPR,rushingCAR,rushingLONG,rushingTD,rushingYDS,rushingYPC
46,2001,62,Derrick Johnson,Waco,Texas,OLB,76.0,205.0,4,0.9779,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [234]:
recruit_and_season_avg_df.isna().sum()

year                      0
ranking                   0
player                    0
school                    0
team                      0
position                  0
height                   33
weight                   35
stars                     0
rating                    0
player_id                 0
interceptionsAVG       7118
interceptionsINT       7118
interceptionsTD        7118
interceptionsYDS       7118
kickReturnsAVG         8229
kickReturnsLONG        8229
kickReturnsNO          8229
kickReturnsTD          8229
kickReturnsYDS         8229
kickingFGA            10156
kickingFGM            10156
kickingLONG           10156
kickingPCT            10156
kickingPTS            10156
kickingXPA            10156
kickingXPM            10156
passingATT             8849
passingCOMPLETIONS     8849
passingINT             8849
passingPCT             8849
passingTD              8849
passingYDS             8849
passingYPA             8849
puntReturnsAVG         8583
puntReturnsLONG     

In [ ]:
nba["College"].fillna("No College", inplace = True) 

In [240]:
recruit_and_season_avg_df['height'].fillna(recruit_and_season_avg_df['height'].mean(), inplace=True)

In [241]:
recruit_and_season_avg_df.isna().sum()

year                      0
ranking                   0
player                    0
school                    0
team                      0
position                  0
height                    0
weight                   35
stars                     0
rating                    0
player_id                 0
interceptionsAVG       7118
interceptionsINT       7118
interceptionsTD        7118
interceptionsYDS       7118
kickReturnsAVG         8229
kickReturnsLONG        8229
kickReturnsNO          8229
kickReturnsTD          8229
kickReturnsYDS         8229
kickingFGA            10156
kickingFGM            10156
kickingLONG           10156
kickingPCT            10156
kickingPTS            10156
kickingXPA            10156
kickingXPM            10156
passingATT             8849
passingCOMPLETIONS     8849
passingINT             8849
passingPCT             8849
passingTD              8849
passingYDS             8849
passingYPA             8849
puntReturnsAVG         8583
puntReturnsLONG     

In [242]:
recruit_and_season_avg_df['weight'].fillna(recruit_and_season_avg_df['weight'].mean(), inplace=True)

In [243]:
recruit_and_season_avg_df.isna().sum()

year                      0
ranking                   0
player                    0
school                    0
team                      0
position                  0
height                    0
weight                    0
stars                     0
rating                    0
player_id                 0
interceptionsAVG       7118
interceptionsINT       7118
interceptionsTD        7118
interceptionsYDS       7118
kickReturnsAVG         8229
kickReturnsLONG        8229
kickReturnsNO          8229
kickReturnsTD          8229
kickReturnsYDS         8229
kickingFGA            10156
kickingFGM            10156
kickingLONG           10156
kickingPCT            10156
kickingPTS            10156
kickingXPA            10156
kickingXPM            10156
passingATT             8849
passingCOMPLETIONS     8849
passingINT             8849
passingPCT             8849
passingTD              8849
passingYDS             8849
passingYPA             8849
puntReturnsAVG         8583
puntReturnsLONG     

In [244]:
recruit_and_season_avg_df.fillna(0, inplace=True)

In [245]:
recruit_and_season_avg_df.isna().sum()

year                  0
ranking               0
player                0
school                0
team                  0
position              0
height                0
weight                0
stars                 0
rating                0
player_id             0
interceptionsAVG      0
interceptionsINT      0
interceptionsTD       0
interceptionsYDS      0
kickReturnsAVG        0
kickReturnsLONG       0
kickReturnsNO         0
kickReturnsTD         0
kickReturnsYDS        0
kickingFGA            0
kickingFGM            0
kickingLONG           0
kickingPCT            0
kickingPTS            0
kickingXPA            0
kickingXPM            0
passingATT            0
passingCOMPLETIONS    0
passingINT            0
passingPCT            0
passingTD             0
passingYDS            0
passingYPA            0
puntReturnsAVG        0
puntReturnsLONG       0
puntReturnsNO         0
puntReturnsTD         0
puntReturnsYDS        0
puntingIn 20          0
puntingLONG           0
puntingNO       

In [263]:
Combine_df = pd.read_excel('Draft_2001_2015.xls')

In [264]:
Combine_df.head()

,Year,Player,Pos,Age,AV,School,Unnamed: 6,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)▲
0,2015,Jameis Winston\WinsJa00,QB,21,59,Florida State,College Stats,2021-06-04,231,4.97,28.5,NaN,103.0,7.16,4.36,Tampa Bay Buccaneers / 1st / 1st pick / 2015
1,2015,Marcus Mariota\MariMa01,QB,21,49,Oregon,College Stats,2021-06-04,222,4.52,36.0,NaN,121.0,6.87,4.11,Tennessee Titans / 1st / 2nd pick / 2015
2,2015,Amari Cooper\CoopAm00,WR,20,51,Alabama,College Stats,2021-06-01,211,4.42,33.0,NaN,120.0,6.71,3.98,Oakland Raiders / 1st / 4th pick / 2015
3,2015,Brandon Scherff\ScheBr00,OT,23,43,Iowa,College Stats,2021-06-05,319,5.05,NaN,23.0,NaN,NaN,NaN,Washington Redskins / 1st / 5th pick / 2015
4,2015,Leonard Williams\WillLe02,DT,20,49,USC,College Stats,2021-06-05,302,4.97,29.5,NaN,106.0,7.59,4.53,New York Jets / 1st / 6th pick / 2015


Looking at the data above we will have to clean up the player name and change the height column back to something usable.
drafted will need be changed to numerical. 

In [265]:
Combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 16 columns):
Year                    3000 non-null int64
Player                  3000 non-null object
Pos                     3000 non-null object
Age                     3000 non-null int64
AV                      3000 non-null int64
School                  3000 non-null object
Unnamed: 6              2460 non-null object
Height                  3000 non-null datetime64[ns]
Wt                      3000 non-null int64
40YD                    2987 non-null float64
Vertical                2344 non-null float64
BenchReps               2088 non-null float64
Broad Jump              2312 non-null float64
3Cone                   1969 non-null float64
Shuttle                 2006 non-null float64
Drafted (tm/rnd/yr)▲    2468 non-null object
dtypes: datetime64[ns](1), float64(6), int64(4), object(5)
memory usage: 375.1+ KB


In [266]:
Combine_df = Combine_df.drop(['Unnamed: 6'], axis=1)

In [267]:
Combine_df['Player'][0]

'Jameis Winston\\WinsJa00'

In [268]:
Combine_df['Player_name'] = Combine_df['Player'].str.split("\\")

In [269]:
Combine_df

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)▲,Player_name
0,2015,Jameis Winston\WinsJa00,QB,21,59,Florida State,2021-06-04,231,4.97,28.5,NaN,103.0,7.16,4.36,Tampa Bay Buccaneers / 1st / 1st pick / 2015,"[Jameis Winston, WinsJa00]"
1,2015,Marcus Mariota\MariMa01,QB,21,49,Oregon,2021-06-04,222,4.52,36.0,NaN,121.0,6.87,4.11,Tennessee Titans / 1st / 2nd pick / 2015,"[Marcus Mariota, MariMa01]"
2,2015,Amari Cooper\CoopAm00,WR,20,51,Alabama,2021-06-01,211,4.42,33.0,NaN,120.0,6.71,3.98,Oakland Raiders / 1st / 4th pick / 2015,"[Amari Cooper, CoopAm00]"
3,2015,Brandon Scherff\ScheBr00,OT,23,43,Iowa,2021-06-05,319,5.05,NaN,23.0,NaN,NaN,NaN,Washington Redskins / 1st / 5th pick / 2015,"[Brandon Scherff, ScheBr00]"
4,2015,Leonard Williams\WillLe02,DT,20,49,USC,2021-06-05,302,4.97,29.5,NaN,106.0,7.59,4.53,New York Jets / 1st / 6th pick / 2015,"[Leonard Williams, WillLe02]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2001,Fred Wakefield\WakeFr00,DE,22,17,Illinois,2021-06-07,287,5.09,29.0,26.0,108.0,7.49,4.64,NaN,"[Fred Wakefield, WakeFr00]"
2996,2001,James Whitley\WhitJa21,CB,21,2,Michigan,2021-05-10,193,4.73,39.0,9.0,118.0,7.04,4.40,NaN,"[James Whitley, WhitJa21]"
2997,2001,Reggie White\WhitRe20,RB,21,0,Oklahoma State,2000-06-01,228,4.58,37.5,17.0,114.0,7.48,4.40,NaN,"[Reggie White, WhitRe20]"
2998,2001,Boo Williams\WillBo00,TE,21,11,Arkansas,2021-06-04,237,4.72,35.5,NaN,119.0,7.10,4.33,NaN,"[Boo Williams, WillBo00]"


In [270]:
Combine_df['Player_name'][0][0]

'Jameis Winston'

In [271]:
Combine_df[['Player_names','Garbage']] = pd.DataFrame(Combine_df.Player_name.tolist(), index= Combine_df.index)

In [272]:
Combine_df

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)▲,Player_name,Player_names,Garbage
0,2015,Jameis Winston\WinsJa00,QB,21,59,Florida State,2021-06-04,231,4.97,28.5,NaN,103.0,7.16,4.36,Tampa Bay Buccaneers / 1st / 1st pick / 2015,"[Jameis Winston, WinsJa00]",Jameis Winston,WinsJa00
1,2015,Marcus Mariota\MariMa01,QB,21,49,Oregon,2021-06-04,222,4.52,36.0,NaN,121.0,6.87,4.11,Tennessee Titans / 1st / 2nd pick / 2015,"[Marcus Mariota, MariMa01]",Marcus Mariota,MariMa01
2,2015,Amari Cooper\CoopAm00,WR,20,51,Alabama,2021-06-01,211,4.42,33.0,NaN,120.0,6.71,3.98,Oakland Raiders / 1st / 4th pick / 2015,"[Amari Cooper, CoopAm00]",Amari Cooper,CoopAm00
3,2015,Brandon Scherff\ScheBr00,OT,23,43,Iowa,2021-06-05,319,5.05,NaN,23.0,NaN,NaN,NaN,Washington Redskins / 1st / 5th pick / 2015,"[Brandon Scherff, ScheBr00]",Brandon Scherff,ScheBr00
4,2015,Leonard Williams\WillLe02,DT,20,49,USC,2021-06-05,302,4.97,29.5,NaN,106.0,7.59,4.53,New York Jets / 1st / 6th pick / 2015,"[Leonard Williams, WillLe02]",Leonard Williams,WillLe02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2001,Fred Wakefield\WakeFr00,DE,22,17,Illinois,2021-06-07,287,5.09,29.0,26.0,108.0,7.49,4.64,NaN,"[Fred Wakefield, WakeFr00]",Fred Wakefield,WakeFr00
2996,2001,James Whitley\WhitJa21,CB,21,2,Michigan,2021-05-10,193,4.73,39.0,9.0,118.0,7.04,4.40,NaN,"[James Whitley, WhitJa21]",James Whitley,WhitJa21
2997,2001,Reggie White\WhitRe20,RB,21,0,Oklahoma State,2000-06-01,228,4.58,37.5,17.0,114.0,7.48,4.40,NaN,"[Reggie White, WhitRe20]",Reggie White,WhitRe20
2998,2001,Boo Williams\WillBo00,TE,21,11,Arkansas,2021-06-04,237,4.72,35.5,NaN,119.0,7.10,4.33,NaN,"[Boo Williams, WillBo00]",Boo Williams,WillBo00


In [273]:
Combine_df = Combine_df.drop(['Player_name'], axis=1)


In [274]:
Combine_df.head()

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)▲,Player_names,Garbage
0,2015,Jameis Winston\WinsJa00,QB,21,59,Florida State,2021-06-04,231,4.97,28.5,NaN,103.0,7.16,4.36,Tampa Bay Buccaneers / 1st / 1st pick / 2015,Jameis Winston,WinsJa00
1,2015,Marcus Mariota\MariMa01,QB,21,49,Oregon,2021-06-04,222,4.52,36.0,NaN,121.0,6.87,4.11,Tennessee Titans / 1st / 2nd pick / 2015,Marcus Mariota,MariMa01
2,2015,Amari Cooper\CoopAm00,WR,20,51,Alabama,2021-06-01,211,4.42,33.0,NaN,120.0,6.71,3.98,Oakland Raiders / 1st / 4th pick / 2015,Amari Cooper,CoopAm00
3,2015,Brandon Scherff\ScheBr00,OT,23,43,Iowa,2021-06-05,319,5.05,NaN,23.0,NaN,NaN,NaN,Washington Redskins / 1st / 5th pick / 2015,Brandon Scherff,ScheBr00
4,2015,Leonard Williams\WillLe02,DT,20,49,USC,2021-06-05,302,4.97,29.5,NaN,106.0,7.59,4.53,New York Jets / 1st / 6th pick / 2015,Leonard Williams,WillLe02


In [275]:
boolean = Combine_df['Garbage'].duplicated().any()

In [276]:
boolean

True

In [277]:
duplicateRowsgarbageDF = Combine_df[Combine_df.Garbage.duplicated(keep=False)]

In [278]:
duplicateRowsgarbageDF

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)▲,Player_names,Garbage
1658,2007,Buster Davis\DaviBu99,WR,23,2,Florida State,2021-06-01,207,4.41,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals / 3rd / 69th pick / 2007,Buster Davis,DaviBu99
1659,2007,Buster Davis\DaviBu99,ILB,23,2,Florida State,2021-05-09,239,4.64,31.0,24.0,104.0,7.28,4.37,Arizona Cardinals / 3rd / 69th pick / 2007,Buster Davis,DaviBu99


In [279]:
Combine_df.drop_duplicates(subset=['Garbage'],keep='last', inplace=True)

In [280]:
duplicatedrowsplayer_namesDF = Combine_df[Combine_df.Player_names.duplicated(keep=False)]

In [281]:
duplicatedrowsplayer_namesDF

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)▲,Player_names,Garbage
75,2015,David Johnson\JohnDa08,RB,23,42,New Mexico,2021-06-01,224,4.50,41.5,25.0,127.0,6.82,4.27,Arizona Cardinals / 3rd / 86th pick / 2015,David Johnson,JohnDa08
83,2015,Matt Jones\JoneMa04,RB,21,10,Florida,2021-06-02,231,4.61,31.5,20.0,112.0,NaN,4.20,Washington Redskins / 3rd / 95th pick / 2015,Matt Jones,JoneMa04
104,2015,Josh Shaw\ShawJo00,CB,22,7,USC,2000-06-01,201,4.44,37.5,26.0,130.0,7.01,4.12,Cincinnati Bengals / 4th / 120th pick / 2015,Josh Shaw,ShawJo00
150,2015,Josh Robinson\RobiJo02,RB,22,0,Mississippi State,2021-05-08,217,4.70,32.0,21.0,113.0,NaN,NaN,Indianapolis Colts / 6th / 205th pick / 2015,Josh Robinson,RobiJo02
182,2015,Taiwan Jones\JoneTa01,ILB,21,0,Michigan State,2021-06-03,245,4.95,32.0,19.0,115.0,7.25,4.33,NaN,Taiwan Jones,JoneTa01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,2002,Adrian Peterson\PeteAd00,RB,22,12,Georgia Southern,2021-05-10,214,4.68,34.0,14.0,119.0,7.59,4.60,Chicago Bears / 6th / 199th pick / 2002,Adrian Peterson,PeteAd00
2762,2002,Steve Smith\SmitSt21,S,22,0,Oregon,2021-06-01,193,4.62,30.5,NaN,110.0,7.00,4.22,Jacksonville Jaguars / 7th / 246th pick / 2002,Steve Smith,SmitSt21
2791,2002,Josh Norman\NormJo00,TE,21,3,Oklahoma,2021-06-02,236,4.65,32.0,18.0,115.0,NaN,NaN,NaN,Josh Norman,NormJo00
2864,2001,Steve Smith\SmitSt01,WR,21,139,Utah,2021-05-09,184,4.41,38.5,NaN,121.0,7.44,4.25,Carolina Panthers / 3rd / 74th pick / 2001,Steve Smith,SmitSt01


In [282]:
duplicatedrowsplayer_namesDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 75 to 2941
Data columns (total 17 columns):
Year                    65 non-null int64
Player                  65 non-null object
Pos                     65 non-null object
Age                     65 non-null int64
AV                      65 non-null int64
School                  65 non-null object
Height                  65 non-null datetime64[ns]
Wt                      65 non-null int64
40YD                    65 non-null float64
Vertical                51 non-null float64
BenchReps               42 non-null float64
Broad Jump              50 non-null float64
3Cone                   40 non-null float64
Shuttle                 41 non-null float64
Drafted (tm/rnd/yr)▲    59 non-null object
Player_names            65 non-null object
Garbage                 65 non-null object
dtypes: datetime64[ns](1), float64(6), int64(4), object(6)
memory usage: 9.1+ KB


In [283]:
duplicatedcombineDF = Combine_df[Combine_df.duplicated(['Player_names', 'School'],keep=False)]

In [284]:
duplicatedcombineDF

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)▲,Player_names,Garbage
2011,2005,Derrick Johnson\JohnDe25,OLB,22,107,Texas,2021-06-03,242,4.52,37.5,NaN,120.0,7.20,3.87,Kansas City Chiefs / 1st / 15th pick / 2005,Derrick Johnson,JohnDe25
2147,2005,Derrick Johnson\JohnDe24,CB,23,2,Texas,2021-05-11,197,4.43,NaN,18.0,NaN,7.15,4.10,San Francisco 49ers / 6th / 205th pick / 2005,Derrick Johnson,JohnDe24


In [285]:
Combine_df.drop_duplicates(subset=['Player_names', 'School'],keep='first', inplace=True) #the other guy isn't in our other DF

In [286]:
Combine_df.isna().sum()

Year                       0
Player                     0
Pos                        0
Age                        0
AV                         0
School                     0
Height                     0
Wt                         0
40YD                      13
Vertical                 654
BenchReps                911
Broad Jump               686
3Cone                   1030
Shuttle                  993
Drafted (tm/rnd/yr)▲     532
Player_names               0
Garbage                    0
dtype: int64

In [287]:
Combine_df.describe()

,Year,Age,AV,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle
count,2998.000000,2998.000000,2998.000000,2998.000000,2985.000000,2344.000000,2087.000000,2312.000000,1968.000000,2005.000000
mean,2008.001334,22.090394,20.307872,245.037358,4.763548,33.265572,21.644945,114.439014,7.257464,4.366933
std,4.322269,0.969075,27.676770,45.108388,0.301496,4.224971,6.545581,9.143346,0.402459,0.261408
min,2001.000000,18.000000,-4.000000,156.000000,4.220000,19.500000,2.000000,83.000000,6.420000,3.730000
25%,2004.000000,22.000000,1.000000,207.000000,4.520000,30.500000,17.000000,109.000000,6.960000,4.170000
50%,2008.000000,22.000000,8.000000,237.000000,4.690000,33.500000,21.000000,115.000000,7.180000,4.330000
75%,2012.000000,23.000000,30.000000,285.000000,4.970000,36.000000,26.000000,121.000000,7.510000,4.530000
max,2015.000000,28.000000,276.000000,375.000000,5.990000,46.000000,49.000000,147.000000,9.000000,5.270000


the 40, shuttle and 3cone have a small std so I feel confident in giving them the avg for their nan values

In [288]:
Combine_df['40YD'].fillna(Combine_df['40YD'].mean(), inplace=True)
Combine_df['3Cone'].fillna(Combine_df['3Cone'].mean(), inplace=True)
Combine_df['Shuttle'].fillna(Combine_df['Shuttle'].mean(), inplace=True)

In [290]:
Combine_df['Drafted (tm/rnd/yr)▲'][0]

'Tampa Bay Buccaneers / 1st / 1st pick / 2015'

In [291]:
Combine_df.rename(columns = {'Drafted (tm/rnd/yr)▲':'Drafted'}, inplace = True)

In [293]:
Combine_df['Drafted'].fillna('None / 8th / 300th pick / 2020', inplace=True)

In [294]:
Combine_df['Drafted'] = Combine_df['Drafted'].str.split("/")

In [295]:
Combine_df

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Drafted,Player_names,Garbage
0,2015,Jameis Winston\WinsJa00,QB,21,59,Florida State,2021-06-04,231,4.97,28.5,NaN,103.0,7.160000,4.360000,"[Tampa Bay Buccaneers , 1st , 1st pick , 2015]",Jameis Winston,WinsJa00
1,2015,Marcus Mariota\MariMa01,QB,21,49,Oregon,2021-06-04,222,4.52,36.0,NaN,121.0,6.870000,4.110000,"[Tennessee Titans , 1st , 2nd pick , 2015]",Marcus Mariota,MariMa01
2,2015,Amari Cooper\CoopAm00,WR,20,51,Alabama,2021-06-01,211,4.42,33.0,NaN,120.0,6.710000,3.980000,"[Oakland Raiders , 1st , 4th pick , 2015]",Amari Cooper,CoopAm00
3,2015,Brandon Scherff\ScheBr00,OT,23,43,Iowa,2021-06-05,319,5.05,NaN,23.0,NaN,7.257464,4.366933,"[Washington Redskins , 1st , 5th pick , 2015]",Brandon Scherff,ScheBr00
4,2015,Leonard Williams\WillLe02,DT,20,49,USC,2021-06-05,302,4.97,29.5,NaN,106.0,7.590000,4.530000,"[New York Jets , 1st , 6th pick , 2015]",Leonard Williams,WillLe02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2001,Fred Wakefield\WakeFr00,DE,22,17,Illinois,2021-06-07,287,5.09,29.0,26.0,108.0,7.490000,4.640000,"[None , 8th , 300th pick , 2020]",Fred Wakefield,WakeFr00
2996,2001,James Whitley\WhitJa21,CB,21,2,Michigan,2021-05-10,193,4.73,39.0,9.0,118.0,7.040000,4.400000,"[None , 8th , 300th pick , 2020]",James Whitley,WhitJa21
2997,2001,Reggie White\WhitRe20,RB,21,0,Oklahoma State,2000-06-01,228,4.58,37.5,17.0,114.0,7.480000,4.400000,"[None , 8th , 300th pick , 2020]",Reggie White,WhitRe20
2998,2001,Boo Williams\WillBo00,TE,21,11,Arkansas,2021-06-04,237,4.72,35.5,NaN,119.0,7.100000,4.330000,"[None , 8th , 300th pick , 2020]",Boo Williams,WillBo00


In [296]:
Combine_df[['Drafted_team','Round','Pick','year_d']] = pd.DataFrame(Combine_df.Drafted.tolist(), index= Combine_df.index)

In [297]:
Combine_df

,Year,Player,Pos,Age,AV,School,Height,Wt,40YD,Vertical,...,Broad Jump,3Cone,Shuttle,Drafted,Player_names,Garbage,Drafted_team,Round,Pick,year_d
0,2015,Jameis Winston\WinsJa00,QB,21,59,Florida State,2021-06-04,231,4.97,28.5,...,103.0,7.160000,4.360000,"[Tampa Bay Buccaneers , 1st , 1st pick , 2015]",Jameis Winston,WinsJa00,Tampa Bay Buccaneers,1st,1st pick,2015
1,2015,Marcus Mariota\MariMa01,QB,21,49,Oregon,2021-06-04,222,4.52,36.0,...,121.0,6.870000,4.110000,"[Tennessee Titans , 1st , 2nd pick , 2015]",Marcus Mariota,MariMa01,Tennessee Titans,1st,2nd pick,2015
2,2015,Amari Cooper\CoopAm00,WR,20,51,Alabama,2021-06-01,211,4.42,33.0,...,120.0,6.710000,3.980000,"[Oakland Raiders , 1st , 4th pick , 2015]",Amari Cooper,CoopAm00,Oakland Raiders,1st,4th pick,2015
3,2015,Brandon Scherff\ScheBr00,OT,23,43,Iowa,2021-06-05,319,5.05,NaN,...,NaN,7.257464,4.366933,"[Washington Redskins , 1st , 5th pick , 2015]",Brandon Scherff,ScheBr00,Washington Redskins,1st,5th pick,2015
4,2015,Leonard Williams\WillLe02,DT,20,49,USC,2021-06-05,302,4.97,29.5,...,106.0,7.590000,4.530000,"[New York Jets , 1st , 6th pick , 2015]",Leonard Williams,WillLe02,New York Jets,1st,6th pick,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2001,Fred Wakefield\WakeFr00,DE,22,17,Illinois,2021-06-07,287,5.09,29.0,...,108.0,7.490000,4.640000,"[None , 8th , 300th pick , 2020]",Fred Wakefield,WakeFr00,None,8th,300th pick,2020
2996,2001,James Whitley\WhitJa21,CB,21,2,Michigan,2021-05-10,193,4.73,39.0,...,118.0,7.040000,4.400000,"[None , 8th , 300th pick , 2020]",James Whitley,WhitJa21,None,8th,300th pick,2020
2997,2001,Reggie White\WhitRe20,RB,21,0,Oklahoma State,2000-06-01,228,4.58,37.5,...,114.0,7.480000,4.400000,"[None , 8th , 300th pick , 2020]",Reggie White,WhitRe20,None,8th,300th pick,2020
2998,2001,Boo Williams\WillBo00,TE,21,11,Arkansas,2021-06-04,237,4.72,35.5,...,119.0,7.100000,4.330000,"[None , 8th , 300th pick , 2020]",Boo Williams,WillBo00,None,8th,300th pick,2020


In [298]:
Combine_df.groupby(['Pos'])['Vertical','BenchReps','Broad Jump'].mean()

,Vertical,BenchReps,Broad Jump
Pos,,,
C,28.463235,26.743243,101.911765
CB,36.364815,14.667969,122.106618
DE,33.507732,24.376884,115.256545
DT,29.475410,28.864865,104.692737
FB,32.363636,23.500000,112.490909
ILB,33.530435,23.438596,114.754386
K,33.500000,13.000000,116.000000
LS,NaN,15.000000,NaN
OG,28.127660,26.136986,101.170370


In [299]:
Combine_df['Vertical']=Combine_df.groupby(['Pos'])['Vertical'].apply(lambda x : x.fillna(x.mean()))

In [302]:
Combine_df['BenchReps']=Combine_df.groupby(['Pos'])['BenchReps'].apply(lambda x : x.fillna(x.mean()))
Combine_df['Broad Jump']=Combine_df.groupby(['Pos'])['Broad Jump'].apply(lambda x : x.fillna(x.mean()))

In [303]:
Combine_df.isna().sum()

Year            0
Player          0
Pos             0
Age             0
AV              0
School          0
Height          0
Wt              0
40YD            0
Vertical        5
BenchReps       0
Broad Jump      5
3Cone           0
Shuttle         0
Drafted         0
Player_names    0
Garbage         0
Drafted_team    0
Round           0
Pick            0
year_d          0
dtype: int64

In [304]:
Combine_df.fillna(0, inplace=True)

In [305]:
Combine_df.isna().sum()

Year            0
Player          0
Pos             0
Age             0
AV              0
School          0
Height          0
Wt              0
40YD            0
Vertical        0
BenchReps       0
Broad Jump      0
3Cone           0
Shuttle         0
Drafted         0
Player_names    0
Garbage         0
Drafted_team    0
Round           0
Pick            0
year_d          0
dtype: int64

In [307]:
Combine_df['Round'] = Combine_df.Round.str.extract('(\d+)')

In [309]:
Combine_df['Pick'] = Combine_df.Pick.str.extract('(\d+)')

In [310]:
Combine_df.rename(columns = {'School':'team'}, inplace = True)

In [311]:
Combine_df.rename(columns = {'Player':'Player_extra'}, inplace = True)

In [312]:
Combine_df.rename(columns = {'Player_names':'player'}, inplace = True)

In [313]:
Combine_df

,Year,Player_extra,Pos,Age,AV,team,Height,Wt,40YD,Vertical,...,Broad Jump,3Cone,Shuttle,Drafted,player,Garbage,Drafted_team,Round,Pick,year_d
0,2015,Jameis Winston\WinsJa00,QB,21,59,Florida State,2021-06-04,231,4.97,28.500000,...,103.000000,7.160000,4.360000,"[Tampa Bay Buccaneers , 1st , 1st pick , 2015]",Jameis Winston,WinsJa00,Tampa Bay Buccaneers,1,1,2015
1,2015,Marcus Mariota\MariMa01,QB,21,49,Oregon,2021-06-04,222,4.52,36.000000,...,121.000000,6.870000,4.110000,"[Tennessee Titans , 1st , 2nd pick , 2015]",Marcus Mariota,MariMa01,Tennessee Titans,1,2,2015
2,2015,Amari Cooper\CoopAm00,WR,20,51,Alabama,2021-06-01,211,4.42,33.000000,...,120.000000,6.710000,3.980000,"[Oakland Raiders , 1st , 4th pick , 2015]",Amari Cooper,CoopAm00,Oakland Raiders,1,4,2015
3,2015,Brandon Scherff\ScheBr00,OT,23,43,Iowa,2021-06-05,319,5.05,28.357143,...,102.844311,7.257464,4.366933,"[Washington Redskins , 1st , 5th pick , 2015]",Brandon Scherff,ScheBr00,Washington Redskins,1,5,2015
4,2015,Leonard Williams\WillLe02,DT,20,49,USC,2021-06-05,302,4.97,29.500000,...,106.000000,7.590000,4.530000,"[New York Jets , 1st , 6th pick , 2015]",Leonard Williams,WillLe02,New York Jets,1,6,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2001,Fred Wakefield\WakeFr00,DE,22,17,Illinois,2021-06-07,287,5.09,29.000000,...,108.000000,7.490000,4.640000,"[None , 8th , 300th pick , 2020]",Fred Wakefield,WakeFr00,None,8,300,2020
2996,2001,James Whitley\WhitJa21,CB,21,2,Michigan,2021-05-10,193,4.73,39.000000,...,118.000000,7.040000,4.400000,"[None , 8th , 300th pick , 2020]",James Whitley,WhitJa21,None,8,300,2020
2997,2001,Reggie White\WhitRe20,RB,21,0,Oklahoma State,2000-06-01,228,4.58,37.500000,...,114.000000,7.480000,4.400000,"[None , 8th , 300th pick , 2020]",Reggie White,WhitRe20,None,8,300,2020
2998,2001,Boo Williams\WillBo00,TE,21,11,Arkansas,2021-06-04,237,4.72,35.500000,...,119.000000,7.100000,4.330000,"[None , 8th , 300th pick , 2020]",Boo Williams,WillBo00,None,8,300,2020


In [314]:
Combine_season_recruit_df = pd.merge(Combine_df, recruit_and_season_avg_df, on=["player", "team"])

In [318]:
Combine_season_recruit_df = Combine_season_recruit_df.drop(['Player_extra','Year','Drafted'], axis=1)

In [321]:
Combine_season_recruit_df.columns

Index(['Pos', 'Age', 'AV', 'team', 'Height', 'Wt', '40YD', 'Vertical',
       'BenchReps', 'Broad Jump', '3Cone', 'Shuttle', 'player', 'Garbage',
       'Drafted_team', 'Round', 'Pick', 'year_d', 'year', 'ranking', 'school',
       'position', 'height', 'weight', 'stars', 'rating', 'player_id',
       'interceptionsAVG', 'interceptionsINT', 'interceptionsTD',
       'interceptionsYDS', 'kickReturnsAVG', 'kickReturnsLONG',
       'kickReturnsNO', 'kickReturnsTD', 'kickReturnsYDS', 'kickingFGA',
       'kickingFGM', 'kickingLONG', 'kickingPCT', 'kickingPTS', 'kickingXPA',
       'kickingXPM', 'passingATT', 'passingCOMPLETIONS', 'passingINT',
       'passingPCT', 'passingTD', 'passingYDS', 'passingYPA', 'puntReturnsAVG',
       'puntReturnsLONG', 'puntReturnsNO', 'puntReturnsTD', 'puntReturnsYDS',
       'puntingIn 20', 'puntingLONG', 'puntingNO', 'puntingTB', 'puntingYDS',
       'puntingYPP', 'receivingLONG', 'receivingREC', 'receivingTD',
       'receivingYDS', 'receivingYPR', 'rushing

In [322]:
Combine_season_recruit_df = Combine_season_recruit_df.drop(['Height','weight','year_d'], axis=1)

In [323]:
Combine_season_recruit_df.to_csv('Combine_season_recruit_df.csv')

In [328]:
Draft_prediction_data = pd.concat([Combine_season_recruit_df,pd.get_dummies(Combine_season_recruit_df['team'], 
                                                  prefix='college',
                                                  drop_first=True)],
                                                  axis=1).drop(['team'], axis=1)

In [329]:
Draft_prediction_data = pd.concat([Draft_prediction_data,pd.get_dummies(Draft_prediction_data['Pos'], 
                                                  prefix='Position',
                                                  drop_first=True)],
                                                  axis=1).drop(['Pos'], axis=1)

In [330]:
Draft_prediction_data

,Age,AV,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,player,...,Position_K,Position_OG,Position_OLB,Position_OT,Position_P,Position_QB,Position_RB,Position_S,Position_TE,Position_WR
0,21,59,231,4.97,28.500000,20.833333,103.000000,7.160000,4.360000,Jameis Winston,...,0,0,0,0,0,1,0,0,0,0
1,21,49,222,4.52,36.000000,20.833333,121.000000,6.870000,4.110000,Marcus Mariota,...,0,0,0,0,0,1,0,0,0,0
2,20,51,211,4.42,33.000000,15.277419,120.000000,6.710000,3.980000,Amari Cooper,...,0,0,0,0,0,0,0,0,0,1
3,23,43,319,5.05,28.357143,23.000000,102.844311,7.257464,4.366933,Brandon Scherff,...,0,0,0,1,0,0,0,0,0,0
4,20,49,302,4.97,29.500000,28.864865,106.000000,7.590000,4.530000,Leonard Williams,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,23,1,229,4.46,34.000000,15.277419,116.000000,7.000000,4.366933,Tab Perry,...,0,0,0,0,0,0,0,0,0,1
932,23,0,223,4.59,35.500000,20.000000,111.000000,7.150000,4.140000,Diamond Ferri,...,0,0,0,0,0,0,0,1,0,0
933,22,0,218,4.76,31.500000,22.000000,111.000000,7.170000,4.260000,Matt Grootegoed,...,0,0,0,0,0,0,0,1,0,0
934,23,0,265,5.04,33.340278,18.000000,114.755245,7.257464,4.366933,Alex Holmes,...,0,0,0,0,0,0,0,0,1,0


In [331]:
Draft_prediction_data.columns

Index(['Age', 'AV', 'Wt', '40YD', 'Vertical', 'BenchReps', 'Broad Jump',
       '3Cone', 'Shuttle', 'player',
       ...
       'Position_K', 'Position_OG', 'Position_OLB', 'Position_OT',
       'Position_P', 'Position_QB', 'Position_RB', 'Position_S', 'Position_TE',
       'Position_WR'],
      dtype='object', length=175)

In [332]:
print(Draft_prediction_data.columns.tolist())

['Age', 'AV', 'Wt', '40YD', 'Vertical', 'BenchReps', 'Broad Jump', '3Cone', 'Shuttle', 'player', 'Garbage', 'Drafted_team', 'Round', 'Pick', 'year', 'ranking', 'school', 'position', 'height', 'stars', 'rating', 'player_id', 'interceptionsAVG', 'interceptionsINT', 'interceptionsTD', 'interceptionsYDS', 'kickReturnsAVG', 'kickReturnsLONG', 'kickReturnsNO', 'kickReturnsTD', 'kickReturnsYDS', 'kickingFGA', 'kickingFGM', 'kickingLONG', 'kickingPCT', 'kickingPTS', 'kickingXPA', 'kickingXPM', 'passingATT', 'passingCOMPLETIONS', 'passingINT', 'passingPCT', 'passingTD', 'passingYDS', 'passingYPA', 'puntReturnsAVG', 'puntReturnsLONG', 'puntReturnsNO', 'puntReturnsTD', 'puntReturnsYDS', 'puntingIn 20', 'puntingLONG', 'puntingNO', 'puntingTB', 'puntingYDS', 'puntingYPP', 'receivingLONG', 'receivingREC', 'receivingTD', 'receivingYDS', 'receivingYPR', 'rushingCAR', 'rushingLONG', 'rushingTD', 'rushingYDS', 'rushingYPC', 'college_Arizona', 'college_Arizona State', 'college_Arkansas', 'college_Arkansa

In [333]:
Draft_prediction_data.to_csv('Draft_prediction_data.csv')